# Cloud အတွင်း Data Science: "Azure ML SDK" နည်းလမ်း

## အကျဉ်းချုပ်

ဒီ notebook မှာ Azure ML SDK ကို သုံးပြီး Azure ML မှတဆင့် မော်ဒယ်တစ်ခုကို လေ့ကျင့်၊ တင်သွင်း၊ အသုံးပြုပုံကို လေ့လာသင်ယူပါမယ်။

လိုအပ်ချက်များ:
1. သင်သည် Azure ML workspace တစ်ခုကို ဖန်တီးပြီးဖြစ်ရမည်။
2. သင်သည် [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) ကို Azure ML ထဲသို့ တင်ပြီးဖြစ်ရမည်။
3. သင်သည် ဒီ notebook ကို Azure ML Studio ထဲသို့ တင်ထားပြီးဖြစ်ရမည်။

နောက်ထပ် လုပ်ဆောင်ရမည့်အဆင့်များမှာ:

1. ရှိပြီးသား Workspace အတွင်း Experiment တစ်ခု ဖန်တီးပါ။
2. Compute cluster တစ်ခု ဖန်တီးပါ။
3. Dataset ကို Load လုပ်ပါ။
4. AutoMLConfig ကို အသုံးပြု၍ AutoML ကို Configure လုပ်ပါ။
5. AutoML experiment ကို Run လုပ်ပါ။
6. ရလဒ်များကို စူးစမ်းပြီး အကောင်းဆုံး မော်ဒယ်ကို ရယူပါ။
7. အကောင်းဆုံး မော်ဒယ်ကို Register လုပ်ပါ။
8. အကောင်းဆုံး မော်ဒယ်ကို Deploy လုပ်ပါ။
9. Endpoint ကို အသုံးပြုပါ။

## Azure Machine Learning SDK အတွက် သီးသန့် Import များ


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## အလုပ်ခွင် စတင်ခြင်း  
သိမ်းဆည်းထားသော configuration မှ အလုပ်ခွင် object ကို စတင်ပါ။ config ဖိုင်ကို .\config.json တွင် ရှိနေကြောင်း သေချာပါ။  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML စမ်းသပ်မှုတစ်ခု ဖန်တီးရန်

ကျွန်တော်တို့ အခုမှ စတင်ပြင်ဆင်ထားတဲ့ workspace အတွင်း 'aml-experiment' ဟု အမည်ပေးထားသော စမ်းသပ်မှုတစ်ခု ဖန်တီးကြမယ်။


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## ကွန်ပျူတာကလပ်စတာတစ်ခု ဖန်တီးရန်  
သင့် AutoML အလုပ်လည်ပတ်မှုအတွက် [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) တစ်ခု ဖန်တီးရန် လိုအပ်ပါမည်။  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## ဒေတာ  
သင့် dataset ကို Azure ML တွင် upload လုပ်ပြီးသားဖြစ်ကြောင်းနှင့် key သည် dataset နာမည်နှင့်တူညီကြောင်းသေချာပါ။  


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## အကောင်းဆုံးမော်ဒယ်ကို တင်သွင်းပါ

အောက်ပါကုဒ်ကို အကောင်းဆုံးမော်ဒယ်ကို တင်သွင်းရန် အလုပ်လှန်ပါ။ တင်သွင်းမှုအခြေအနေကို Azure ML ပေါ်တယ်တွင် ကြည့်ရှုနိုင်ပါသည်။ ဒီအဆင့်မှာ မိနစ်အနည်းငယ်ကြာနိုင်ပါသည်။


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Endpoint ကိုသုံးစွဲရန်
အောက်ပါ input နမူနာတွင် input များကို ထည့်သွင်းနိုင်ပါသည်။


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**ဝက်ဘ်ဆိုက်မှတ်ချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ဆိုမှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သိရှိပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတည်သောရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူပညာရှင်များမှ ဘာသာပြန်ဆိုမှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားမှုများ သို့မဟုတ် အဓိပ္ပာယ်မှားမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
